In [1]:
import pandas as pd
import numpy as np

In [2]:
import gensim
# from gensim.scripts.glove2word2vec import glove2word2vec

# glove2word2vec(glove_input_file='../glove.6B.50d.txt',
#               word2vec_output_file='gensim_glove_vectors.txt')

In [3]:
from gensim.models.keyedvectors import KeyedVectors
embedding = KeyedVectors.load_word2vec_format('gensim_glove_vectors.txt')

In [4]:
df = pd.read_csv('clean_data.csv')

In [135]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import accuracy_score

np.random.seed(42)
df_permuted = df.reindex(np.random.permutation(df.index))

In [91]:
embedded = []
for i, row in df_permuted.iterrows():
    text = row['clean_text']
    tokens = text.split(' ')
    emb = []
    for tk in tokens:
        if tk in embedding:
            emb.append(embedding[tk])
#     emb = [embedding[tk] for tk in tokens]
    embedded.append(np.array(emb).mean(axis=0))
embedded = np.array(embedded)

In [93]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import FeatureUnion

In [94]:
vectorizer = TfidfVectorizer()
corpus_tfidf = vectorizer.fit_transform(df_permuted['clean_text'])

In [95]:
svd = TruncatedSVD(1000)
svd_inside = svd.fit_transform(corpus_tfidf)

In [99]:
corpus_tfidf.shape

(2000, 38720)

In [100]:
svd_inside.shape

(2000, 1000)

In [101]:
embedded.shape

(2000, 50)

In [127]:
corpus_tf = np.hstack([svd_inside, embedded, corpus_tfidf.toarray()])

In [129]:
X_train, X_test, y_train, y_test = train_test_split(corpus_tf,
                                                    df_permuted['class'], test_size=.33, random_state=42)

In [131]:
svc = LinearSVC()
svc.fit(X_train, y_train)
accuracy_score(y_test, svc.predict(X_test))

0.8348484848484848

In [136]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
accuracy_score(y_test, lr.predict(X_test))

/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8106060606060606

In [137]:
ridge = RidgeClassifier()
ridge.fit(X_train, y_train)
accuracy_score(y_test, ridge.predict(X_test))

0.8393939393939394

In [71]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization

In [140]:
model = Sequential()
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(.4))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(.4))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(.4))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(.4))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

In [142]:
model.fit(X_train, y_train.values, epochs=5, validation_data=[X_test, y_test.values])

Train on 1340 samples, validate on 660 samples
Epoch 1/5
1340/1340 [==============================] - 3s 2ms/step - loss: 0.0605 - acc: 0.9769 - val_loss: 0.7365 - val_acc: 0.7803
Epoch 2/5
1340/1340 [==============================] - 3s 2ms/step - loss: 0.0605 - acc: 0.9769 - val_loss: 0.6012 - val_acc: 0.8227
Epoch 3/5
1340/1340 [==============================] - 3s 2ms/step - loss: 0.0539 - acc: 0.9776 - val_loss: 0.5891 - val_acc: 0.8258
Epoch 4/5
1340/1340 [==============================] - 3s 2ms/step - loss: 0.0502 - acc: 0.9806 - val_loss: 0.6829 - val_acc: 0.8121
Epoch 5/5
1340/1340 [==============================] - 3s 2ms/step - loss: 0.0521 - acc: 0.9806 - val_loss: 0.6759 - val_acc: 0.8121


In [90]:
accuracy_score(y_test, model.predict_classes(X_test))

0.6803030303030303